In [17]:
import pandas as pd
import numpy as np
import mplfinance as mpl
from tqtklib.file_manager import file_manager


ModuleNotFoundError: No module named 'mplfinance'

# Parameters

In [7]:
FILE_LIST = ["EURJPY_5 Mins_Ask_2024.03.16_2025.03.16.csv", 
             "EURUSD_5 Mins_Ask_2024.03.16_2025.03.16.csv",
            "USDJPY_5 Mins_Ask_2024.03.16_2025.03.16.csv"]

DATA_FOLDER = "../../data/"

# Open DataSets

In [9]:
file_container: dict = {}

In [15]:
data_container: dict = {}
for file_name in FILE_LIST:
    currency = file_name[:6]
    print("Processing", currency)
    data_container[currency] = pd.read_csv(f"{DATA_FOLDER}{file_name}", parse_dates=True, index_col=0)
    
    

Processing EURJPY
Processing EURUSD
Processing USDJPY


In [16]:
data_container["EURJPY"]

,Open,High,Low,Close,Volume
Time (America/Buenos_Aires),,,,,
2024-03-17 18:00:00,162.416,162.416,162.411,162.411,1.80
2024-03-17 18:05:00,162.405,162.426,162.318,162.328,31.65
2024-03-17 18:10:00,162.409,162.409,162.297,162.367,56.00
2024-03-17 18:15:00,162.293,162.340,162.293,162.308,91.62
2024-03-17 18:20:00,162.310,162.462,162.310,162.431,87.21
...,...,...,...,...,...
2025-03-14 17:35:00,161.786,161.799,161.780,161.784,572.10
2025-03-14 17:40:00,161.785,161.788,161.736,161.747,845.70
2025-03-14 17:45:00,161.745,161.748,161.707,161.736,696.60
